## Attempting to add every historical period to the dataset for all currencies and BUs so that we can do the forecast easier.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

In [3]:
input_df_dict = pickle.load(open('../data/processed/all_inputs.p', 'rb'))

df_billings = input_df_dict['billings']
df_cal = input_df_dict['ADBE_cal']
df_bookings = input_df_dict['bookings']
df_FX_fwds  = input_df_dict['FX_forwards']
df_FX_rates = input_df_dict['FX_rates']

In [5]:
all_BU = df_billings['BU'].unique()
all_curr = df_billings['curr'].unique()
all_periods = df_billings['period'].unique()
print('BU length', len(all_BU))
print('curr length', len(all_curr))
print('period length', len(all_periods))

BU length 7
curr length 29
period length 63


In [6]:
7*29*63

12789

In [7]:
len(df_billings)

4096

In [9]:
df_billings.head(40)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,deferred_1M_DC,...,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks
0,ARS,Creative,2019-07,6786.00,155.12,0.0,0.0,0.0,0.0,6.453291e+06,...,9.538759e+05,0.000000,0.0,1.460911e+05,0.000000,0.00,2.156614e+04,0.000000,0.0,4
1,ARS,Creative,2019-08,16472.00,390.02,0.0,0.0,0.0,0.0,7.305797e+06,...,9.631800e+05,0.000000,0.0,1.732747e+05,0.000000,0.00,2.278191e+04,0.000000,0.0,4
2,ARS,Creative,2019-09,19205.00,405.51,0.0,0.0,0.0,0.0,1.065499e+07,...,2.263704e+06,0.000000,0.0,2.176112e+05,0.000000,0.00,4.629725e+04,0.000000,0.0,5
3,ARS,Creative,2019-10,30382.19,532.13,0.0,0.0,0.0,0.0,1.030293e+07,...,1.318347e+06,0.000000,0.0,1.817068e+05,0.000000,0.00,2.337652e+04,0.000000,0.0,4
4,ARS,Creative,2019-11,59049.00,1023.67,0.0,0.0,0.0,0.0,1.134010e+07,...,1.494162e+06,0.000000,0.0,1.960320e+05,0.000000,0.00,2.578235e+04,0.000000,0.0,4
5,ARS,Creative,2019-12,71355.57,1195.94,0.0,0.0,0.0,0.0,1.680997e+07,...,3.154860e+06,0.000000,0.0,2.817820e+05,0.000000,0.00,5.298734e+04,0.000000,0.0,5
6,ARS,Creative,2020-01,79224.95,1323.41,0.0,0.0,0.0,0.0,1.435869e+07,...,9.167059e+05,0.000000,0.0,2.398857e+05,0.000000,0.00,1.528418e+04,0.000000,0.0,4
7,ARS,Creative,2020-02,36596.00,611.42,0.0,0.0,0.0,0.0,1.466749e+07,...,1.337026e+06,0.000000,0.0,2.450065e+05,0.000000,0.00,2.233222e+04,0.000000,0.0,4
8,ARS,Creative,2020-03,49107.00,810.06,0.0,0.0,0.0,0.0,2.163893e+07,...,2.807013e+06,0.000000,0.0,3.555430e+05,0.000000,0.00,4.606905e+04,0.000000,0.0,5
9,AUD,Creative,2015-01,191897.46,160229.26,0.0,0.0,0.0,0.0,3.192462e+06,...,1.907698e+06,29664.800000,0.0,2.664682e+06,0.000000,0.00,1.556240e+06,24592.980000,0.0,4
